## **Описание задачи и датасета**

Датасет Titanic описывает пассажиров парохода «Титаник», затонувшего в 1912 году. Необходимо решить задачу бинарной классификации: по данным о пассажире нужно предсказать, выжил он или нет.

Переменные датасета titanic:

Проект решает задачу бинарной классификации на датасете Titanic: по информации о пассажире нужно предсказать, выжил он в катастрофе или нет (`Survived`: 1 - выжил, 0 - погиб). Данные описывают 891 пассажира лайнера «Титаник» с признаками пола, возраста, класса билета, стоимости билета, каюты и порта посадки.  

Основные поля датасета:

- `PassengerId` - уникальный идентификатор пассажира (служебный);
- `Survived` - целевая переменная (1 - выжил, 0 - погиб);
- `Pclass` - класс билета (1 - высший, 2 - средний, 3 - низший);
- `Name` - полное имя пассажира;
- `Sex` - пол (`male` / `female`);
- `Age` - возраст (часть значений отсутствует);
- `SibSp` - число братьев/сестер и супругов на борту;
- `Parch` - число родителей и детей на борту;
- `Ticket` - номер билета;
- `Fare` - стоимость билета;
- `Cabin` - номер каюты;
- `Embarked` - порт посадки (`S`, `C`, `Q`).

In [842]:
# импорт необходимых библиотек
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os

In [843]:
# загрузка параметров из .env файла
load_dotenv("envs/.params.env", override=True)

TRAIN_DATA_URL = os.getenv("TRAIN_DATA_URL")
TEST_DATA_URL = os.getenv("TEST_DATA_URL")
VAL_SIZE = float(os.getenv("VAL_SIZE"))
EPOCHS = int(os.getenv("EPOCHS"))
HIDDEN_SIZE = int(os.getenv("MODEL_PARAMETER_1"))

In [844]:
# загрузка данных
train_df = pd.read_csv(TRAIN_DATA_URL)
test_df  = pd.read_csv(TEST_DATA_URL)

In [845]:
train_df

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [846]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [847]:
# переименование столбцов датасета train для удобства работы с ними
test_df = test_df.drop(columns=['PassengerId'])
train_df = pd.DataFrame(train_df.values, columns=['Survived'] + list(test_df.columns))

In [848]:
# разделение данных на признаки и целевую переменную
X_train = train_df.drop(columns=['Survived', 'Ticket', 'Name'])
y_train = train_df['Survived']

X_test = test_df.drop(columns=['Ticket', 'Name'])

In [849]:
# Заполнение пропусков - числовые признаки
for X in [X_train, X_test]:
    for col in ['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']:
        if X[col].isna().any():
            X[col] = X[col].fillna(X[col].median())

# Заполнение пропусков - категориальные признаки
for X in [X_train, X_test]:
    for col in ['Embarked', 'Sex', 'Cabin']:
        if X[col].isna().any():
            X[col] = X[col].fillna(X[col].mode()[0])

In [850]:
for X in [X_train, X_test]:
    # бинарное кодирование для Sex признака
    X['Sex'] = (X['Sex'] == 'female').astype(int)

In [851]:
# функция для выделения палубы из номера каюты
def cabin_to_deck(x):
    return x[0]

# применение функции к признаку Cabin
for X in (X_train, X_test):
    for col in ['Embarked', 'Cabin']:
        X['Cabin'] = X['Cabin'].apply(cabin_to_deck)

In [852]:
# метки для категориальных признаков
# применяем к train/test
for col in ['Embarked', 'Cabin']:
    # обучаем кодировщик на train
    le_embarked = LabelEncoder()
    le_embarked.fit(X_train[col])
    for X in [X_train, X_test]:
        X[col] = le_embarked.transform(X[col])

In [853]:
# приведение типов данных
for X in [X_train, X_test]:
    for col in X.columns:
        if X[col].dtypes == 'object':
            X[col] = X[col].astype(type(X[col].unique()[0]))

y_train = y_train.astype(int)

In [854]:
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL_SIZE, random_state=42)